<span style="color:rgb(0, 174, 209); font-size:36px; font-weight: bold;">RED 5R - Premio CERVERA - Guía básica de aplicación</span>

<img src="https://red5r.es/wp-content/uploads/2022/03/Red-5R-Red-Cervera-300x109.png" alt="RED 5R LOGO" style="width: 25%; margin-left: 25%;">

El objetivo de este Notebook es servir cómo guía para los participantes de la competición la cual se encuentra situada dentro de las actividades del proyecto “5R – Red Cervera de tecnologías robóticas en fabricación inteligente”, financiado por el Ministerio de Ciencia e Innovación a través del Centro para el Desarrollo Tecnológico Industrial (CDTI), número de contrato CER-20211007, en el marco del programa “Centros Tecnológicos de Excelencia Cervera”. La red se encuentra formada por los siguientes centros: TEKNIKER, EURECAT, AIMEN, CARTIF y CATEC.

La competición se puede encontrar en el siguiente link de la plataforma Kaggle: [RED 5R - Competition Home Page](https://www.kaggle.com/competitions/5r-competicion/overview)

# 1. Configuración inicial

En este primer paso se importarán las librerías y módulos que serán necesarios en este Notebook.

In [3]:
import os, glob
import numpy as np
import pandas as pd

Además, se incluye un ejemplo de cómo obtener una lista con los nombres de las imágenes "default" tanto del TRAIN como del VALIDATION dataset. Esta metodología se puede aprovechar para obtener la lista de nombres de las máscaras binarias, etc.

In [7]:
# Define the base directories
train_base_dir = '/kaggle/input/5r-competicion/TRAIN/TRAIN/'
valid_base_dir = '/kaggle/input/5r-competicion/VALIDATION/VALIDATION/'

# Function to get image paths containing 'defaultImage' in the name and are PNG files
def get_paths_with_default_image_and_png(base_dir):
    image_paths = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if 'defaultImage' in file and file.lower().endswith('.png'):
                image_paths.append(os.path.join(root, file))
    return image_paths

# Get train_paths and valid_paths
train_paths = get_paths_with_default_image_and_png(train_base_dir)
valid_paths = get_paths_with_default_image_and_png(valid_base_dir)

print(train_paths[0])
print(valid_paths[0])

/kaggle/input/5r-competicion/TRAIN/TRAIN/20240109_122904/20240109_122904_defaultImage.png
/kaggle/input/5r-competicion/VALIDATION/VALIDATION/20240109_123030/20240109_123030_defaultImage.png


# 2. Entrenamiento

A continuación, el siguiente paso consiste entrenar el modelo de aprendizaje automático o aprendizaje profundo. Esta sección incluye la carga de los datos de entrenamiento, los pasos de preprocesamiento, la definición de la arquitectura del modelo y el bucle de entrenamiento. Es recomendable mostrar las métricas relevantes y así como supervisar el progreso del entrenamiento.

In [ ]:
# Include your code here

# 3. Inferencia

Una vez entrenado el modelo, el siguiente paso consiste en obtener las predicciones del modelo sobre un conjunto de datos separado, a menudo el conjunto de prueba o nuevos datos no vistos. En esta sección se incluye la carga de los pesos del modelo entrenado, el preprocesamiento de los datos de prueba, la realización de predicciones y la evaluación del rendimiento del modelo en el conjunto de prueba.

In [ ]:
# Include your code here

### Codificación RLE (Run Length Encoding)

Se incluyen algunas funciones  útiles para codificar las máscaras obtenidas en las predicciones del modelo. Esta codificación de las imágenes es necesaria para generar posteriormente el archivo de entrega. Esta codificación se utiliza para reducir el tamaño del archivo CSV de entrega y para simplificar el proceso de evaluación.

In [ ]:
# Custom function to encode binary mask
def encode_binary_mask(mask: np.ndarray) -> t.Text:
    """Converts a binary mask into OID challenge encoding ascii text."""
    # Check input mask
    if mask.dtype != np.uint8:
        raise ValueError(
            "encode_binary_mask expects a binary mask, received dtype == %s" % mask.dtype
        )

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
            "encode_binary_mask expects a 2d mask, received shape == %s" % mask.shape
        )

    # Convert input mask to expected COCO API input
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # Compress and base64 encoding
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str.decode("utf-8")  # Return as string

# 4. Generar Entrega

El último paso consiste en generar el archivo de entrega denominado `submission.csv` donde se almacenan las predicciones.

El archivo CSV de envío debe tener las siguientes columnas:

- ID: Identificador único de la imagen (también denominado escena).
- Width: anchura de la imagen
- Height: La altura de la imagen
- EncodedMasks: Las máscaras binarias codificadas de la imagen (separadas por espacios)

Es importante asegurarse de que el archivo CSV enviado tiene el formato correcto y de que están presentes todas las columnas requeridas. Cualquier incoherencia en el formato o ausencia de columnas puede dar lugar a errores de evaluación.

Utilice el siguiente código como ejemplo para generar su archivo "submission.csv".

In [8]:
# Generate submission DataFrame
submission_df = pd.DataFrame(columns=['ID', 'Width', 'Height', 'EncodedPixels'])

for i, image_identifier in enumerate(image_files_paths):
    image_id = os.path.basename(image_identifier)
    submission_df.loc[i, 'ID'] = image_id
    height, width = image.shape
    submission_df.loc[i, 'Width'] = width
    submission_df.loc[i, 'Height'] = height
    
    encoded_masks_str = ""
    for binary_mask in image_predictions_masks:
        # Encode the binary mask
        encoded_mask = encode_binary_mask(binary_mask)

        # Append the encoded mask to the string (encoded binary masks are space-separated)
        if encoded_masks_str:
            encoded_masks_str += " " + encoded_mask
        else:
            encoded_masks_str = encoded_mask
            
    submission_df.loc[i, 'EncodedPixels'] = encoded_masks_str
    
# Save submission DataFrame to CSV file
submission_df.to_csv('submission.csv', index=False)

# Display the DataFrame
submission_df.head()

,ID,Width,Height,EncodedPixels
0,12,640,640,AAAA
